In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [48]:
sys.path.append("fastai")

AttributeError: module 'sys' has no attribute 'append'

In [43]:
from niklib.dataset_matify import MattextDataset
from niklib.w2v_model import Word2VecModel
from niklib.dataset_text import TextDataset, TextModelData
from niklib.processor_text import TextProcessor
from niklib.model_text import MaxBiRNN
from falib.core import V, T, to_np
import torch

In [44]:
mattext = MattextDataset.from_pickle("./niklib/train.pickle")


In [40]:
SEQLEN = 50
w2v = Word2VecModel.from_pickle('wiki.no.pickle')
ds = TextDataset.from_pickle('niklib/mattext_ds.pk')
textproc = TextProcessor(w2v, max_pad = ds.get_seqlen())
model = MaxBiRNN(w2v, seqlen=ds.get_seqlen(), n_labels=ds.get_c(),
                 hidden_size=300, n_layers=1, p0=0.2, p1=0.2, p2=0.2, p3=0.2)
model.load_state_dict(torch.load('niklib/MaxBiRNN.pt'))

In [41]:
from falib.core import V
#Predict
texts_test=['The burger Cheesy, original eller spicy. 4 x 80 g. 320 g. Pr kg 155,94', 
            'Coop svenske kjøttboller 400 g. Pr kg 69,00',
            'Barilla spaghetti 1 kg',
            'Barilla tagliatelle M/egg. 500 g. Pr kg 77,00',
            'Lofoten hjemmelaget fiskegrateng 400 g. Pr kg 195,50 600 Pr',
            'Barilla spaghetti fullkorn 1 kg',
            'Coop fra havet laksefilet 500 g. Pr kg 159,80',
            'Barilla saus classico 400 g. Pr kg 59,75',
            'Coop fra havet lettsaltet torskefilet M/skinn. 360 g. Pr kg 147,50',
            'Barilla pesto genovese 190 g. Pr kg 157,37']
xs = textproc.text2features(texts_test)
model.eval()
ys = model(V(xs))
ys = to_np(ys).argmax(axis=1)
labels = [ds.idx2label(y) for y in ys]
labels

Text length: min=4, max=17, mean=9.4
Padding all text to have fixed length = 50


['MeatBalls & Hamburger',
 'MeatBalls & Hamburger',
 'Pasta and Noodle',
 'Take-away meals',
 'Fish Cakes',
 'Pasta and Noodle',
 'Fresh fish',
 'Sauces & Dressing',
 'Fresh fish',
 'Sauces & Dressing']

In [45]:
catids = [mattext.catname2catid(label) for label in labels]

In [46]:
catids

[97, 97, 144, 108, 103, 144, 100, 130, 100, 130]